In [31]:
import os
from io import StringIO

import pandas as pd
import requests
from dotenv import load_dotenv

In [32]:
load_dotenv()
url = os.getenv('MBAJK_URL')
response = requests.get(url)

df = pd.read_json(StringIO(response.text))
df.drop(columns=['contract_name','number'], inplace=True, axis=1)
df['last_update'] = pd.to_datetime(df['last_update'], unit='ms')

df_position = pd.json_normalize(df['position'])
df = pd.concat([df, df_position], axis=1)
df.drop(columns=['position'], inplace=True)

df # 2024-04-30 13:46:24

,name,address,banking,bonus,bike_stands,available_bike_stands,available_bikes,status,last_update,lat,lng
0,"VZAJEMNA, VARUH ZDRAVJA - BETNAVSKA C.",Betnavska c. 83,False,False,20,11,9,OPEN,2024-04-30 14:02:40,46.541990,15.638310
1,TELEMACH - GLAVNI TRG - STARI PERON,Glavni trg 20,False,False,20,15,5,OPEN,2024-04-30 14:05:04,46.557328,15.646833
2,NKBM - TRG LEONA ŠTUKLJA,Ulica slovenske osamosvojitve 5,False,False,20,12,8,OPEN,2024-04-30 14:02:27,46.558519,15.649072
3,PARTIZANSKA C. - ŽELEZNIŠKA POSTAJA,Partizanska cesta 46-48,False,False,20,15,5,OPEN,2024-04-30 14:05:40,46.561555,15.656920
4,EUROPARK - POBREŠKA C.,Pobreška cesta 18,False,False,21,8,13,OPEN,2024-04-30 14:07:34,46.554703,15.652826
5,STROSSMAYERJEVA UL. - TRŽNICA,Strossmayerjeva ulica 3,False,False,20,11,9,OPEN,2024-04-30 14:06:00,46.558498,15.641460
6,GOSPOSVETSKA C. - TURNERJEVA UL.,Gosposvetska cesta 69,False,False,22,18,4,OPEN,2024-04-30 13:58:09,46.562695,15.629350
7,KOROŠKA C. - KOROŠKI VETER,Koroška cesta 74,False,False,20,14,6,OPEN,2024-04-30 14:07:15,46.559319,15.634980
8,GOSPOSVETSKA C. - III. GIMNAZIJA,Gosposvetska cesta 4,False,False,20,17,3,OPEN,2024-04-30 13:58:06,46.560761,15.640530
9,GORKEGA UL. - OŠ FRANCETA PREŠERNA,Gorkega ulica 32,False,False,20,10,10,OPEN,2024-04-30 14:07:33,46.550683,15.639333


In [33]:
for name, group in df.groupby('name'):
    filename = f'../data/raw/{name}.csv'
    if os.path.exists(filename):
        existing_df = pd.read_csv(filename, parse_dates=['last_update'])
        if existing_df['last_update'].max() < group['last_update'].max():
            group.to_csv(filename, mode='a', header=False, index=False)
    else:
        group.to_csv(filename, index=False)